### - 카카오 API : 네비게이션 검색

- 카카오 API 테스트
  - 앱 ID : 
  - 네이티브 앱 키 : 
  - REST API 키 : 
  - JavaScript 키 : 
  - Admin 키 : 
- 키워드 검색
  - 질의어를 통해 장소 검색 결과 반환
  - 네비게이션 길안내 결과 반환

- origin : 33.248678, 126.56734
- destiantion : 33.291055, 126.461936
- 한라산 : 33.361204769954284, 126.53111502985787
---
- 경로 탐색 실패하는 경우 발생 : **출발지, 도착지 주변에 도로가 없는 경우**

#### 1. 경로 검색

In [ ]:
import json
import requests
# REST 키
rest_api_key = '63d0926cf9b14de298157081ba8a8d02'
# 헤더
headers = {"Authorization" : "KakaoAK {}".format(rest_api_key)}
# 파라미터
url1 = "https://apis-navi.kakaomobility.com/v1/directions?origin=126.56734,33.248678&destination=126.461936,33.291055&waypoints=&priority=RECOMMEND&car_fuel=GASOLINE&car_hipass=false&alternatives=false&road_details=false"
url2 = "https://apis-navi.kakaomobility.com/v1/directions?origin=126.56734,33.248678&destination=126.53111502985787,33.361204769954284&waypoints=&priority=RECOMMEND&car_fuel=GASOLINE&car_hipass=false&alternatives=false&road_details=false"

# GET을 이용하여 획득
res1 = requests.get(url1, headers=headers)
# Json을 이용하여 해제
doc1 = json.loads(res1.text)

res2 = requests.get(url2, headers=headers)
doc2 = json.loads(res2.text)

# 200일 경우 정상
res1.status_code, res2.status_code

In [72]:
# 결과(Dict형으로 표현)
print(doc1['routes'][0]['result_code'], doc1['routes'][0]['result_msg'],)
doc1['routes'][0]['summary']

0 길찾기 성공


{'origin': {'name': '', 'x': 126.56733334860414, 'y': 33.24867189296654},
 'destination': {'name': '', 'x': 126.46192692717538, 'y': 33.29105000445024},
 'waypoints': [],
 'priority': 'RECOMMEND',
 'bound': {'min_x': 126.46225482364076,
  'min_y': 33.24724561321495,
  'max_x': 126.56944343869672,
  'max_y': 33.29365953466207},
 'fare': {'taxi': 13800, 'toll': 0},
 'distance': 15004,
 'duration': 1522}

In [70]:
doc2

{'trans_id': '05b61c85169846c38177fe407d6ded62',
 'routes': [{'result_code': 103, 'result_msg': '도착지점 주변의 도로를 탐색할 수 없음'}]}

In [57]:
vertexes = []
for road in doc1['routes'][0]['sections'][0]['roads']:
    test_v = road['vertexes']
    for i in range(0, len(test_v), 2):
        vertexes.append((test_v[i+1], test_v[i]))

In [58]:
vertexes[:10]

[(33.24864534401754, 126.567376703514),
 (33.24864534401754, 126.567376703514),
 (33.248590268564406, 126.56729175890251),
 (33.248590268564406, 126.56729175890251),
 (33.24851740006151, 126.56644524610292),
 (33.24851740006151, 126.56644524610292),
 (33.24848565644762, 126.56603800926246),
 (33.24848565644762, 126.56603800926246),
 (33.249097401159204, 126.56592071290774),
 (33.249097401159204, 126.56592071290774)]

#### 2. 경로 표현

In [59]:
import folium
import pandas as pd

origin = doc1['routes'][0]['summary']['origin']
destination = doc1['routes'][0]['summary']['destination']

map = folium.Map(location=[origin['y'], origin['x']], zoom_start=9)
folium.Marker(location=[origin['y'], origin['x']], popup='origin').add_to(map)
folium.Marker(location=[destination['y'], destination['x']], popup='destination').add_to(map)
folium.PolyLine(vertexes, color='red').add_to(map)
map

### - 데이터 확인

#### 1. POI 정보

In [1]:
import pandas as pd

poi_df = pd.read_excel('./data/220112/POI_장소처리_del.xlsx')

In [2]:
poi_df.head()

,위치좌표 X축값,위치좌표 Y축값,구분,장소명,소재지,Additional_Name
0,126.167211,33.324055,관광지,103순례자의집,제주특별자치도 제주시 한경면 용수리 4110-1,.
1,126.462359,33.357365,관광지,1100고지,제주특별자치도 서귀포시 색달동 산 1-3,천백고지
2,126.459185,33.333267,관광지,1100고지드라이브코스,제주특별자치도 서귀포시 중문동 산 1-2,천백고지드라이브코스
3,126.461936,33.291055,관광지,1100도로드라이브코스,제주특별자치도 서귀포시 하원동 산 72-2,.
4,126.368402,33.477716,관광지,1박2일,제주특별자치도 제주시 애월읍 신엄리 964,.


In [3]:
poi_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2091 entries, 0 to 2090
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   위치좌표 X축값         2091 non-null   float64
 1   위치좌표 Y축값         2091 non-null   float64
 2   구분               2091 non-null   object 
 3   장소명              2091 non-null   object 
 4   소재지              2091 non-null   object 
 5   Additional_Name  2091 non-null   object 
dtypes: float64(2), object(4)
memory usage: 98.1+ KB


In [4]:
poi_df['구분'].value_counts()

숙박            1284
관광지            350
레져/스포츠         240
문화/종교/예술       134
공원/산/동.식물원      83
Name: 구분, dtype: int64

In [5]:
(poi_df['구분'] == '숙박').value_counts()

True     1284
False     807
Name: 구분, dtype: int64

#### 2. 식당 데이터

- 식당 정보

In [ ]:
restaurant_df = pd.read_excel('./data/식당/제주도 식당 정보_전체.xlsx')
restaurant_df.shape

(20096, 39)

In [ ]:
restaurant_df.columns

Index(['상가업소번호', '상호명', '지점명', '상권업종대분류코드', '상권업종대분류명', '상권업종중분류코드',
       '상권업종중분류명', '상권업종소분류코드', '상권업종소분류명', '표준산업분류코드', '표준산업분류명', '시도코드',
       '시도명', '시군구코드', '시군구명', '행정동코드', '행정동명', '법정동코드', '법정동명', '지번코드',
       '대지구분코드', '대지구분명', '지번본번지', '지번부번지', '지번주소', '도로명코드', '도로명', '건물본번지',
       '건물부번지', '건물관리번호', '건물명', '도로명주소', '구우편번호', '신우편번호', '동정보', '층정보',
       '호정보', '경도', '위도'],
      dtype='object')